In [1]:
from stratopy.extractors import cloudsat
from stratopy.extractors import goes
from stratopy.transformers import mergers
from stratopy.transformers import scalers
import os

Download 1 band file from ABI GOES16

In [2]:
goes_obj = goes.GOES16(product_type="ABI-L2-CMIPF", channel=13)
goes_data = goes_obj.fetch("2019 jan 2 18:30")

Lets see one transformer. This one normalizes an ABI image to [0,1].

Returns an xarray with an image of shape (n_bands, image_width, image_height).

In [3]:
norm_goes = scalers.MinMaxNormalize().transform(goes_data)
norm_goes

<xarray.Dataset>
Dimensions:                                 (y: 5424, x: 5424, nbands: 1,
                                             number_of_time_bounds: 2,
                                             number_of_image_bounds: 2, band: 1)
Coordinates:
  * y                                       (y) float64 0.1518 ... -0.1518
  * x                                       (x) float64 -0.1518 ... 0.1518
  * nbands                                  (nbands) int64 1
    t                                       datetime64[ns] ...
    y_image                                 float32 ...
    x_image                                 float32 ...
    band_wavelength                         (band) float32 ...
    band_id                                 (band) int32 ...
Dimensions without coordinates: number_of_time_bounds, number_of_image_bounds,
                                band
Data variables: (12/29)
    CMI                                     (nbands, y, x) float32 nan ... nan
    DQF                                     (y, x) float32 ...
    time_bounds                             (number_of_time_bounds) datetime64[ns] ...
    goes_imager_projection                  int32 ...
    y_image_bounds                          (number_of_image_bounds) float32 ...
    x_image_bounds                          (number_of_image_bounds) float32 ...
    ...                                      ...
    algorithm_dynamic_input_data_container  int32 ...
    percent_uncorrectable_GRB_errors        float32 ...
    percent_uncorrectable_L0_errors         float32 ...
    earth_sun_distance_anomaly_in_AU        float32 ...
    processing_parm_version_container       int32 ...
    algorithm_product_version_container     int32 ...
Attributes: (12/30)
    naming_authority:          gov.nesdis.noaa
    Conventions:               CF-1.7
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    standard_name_vocabulary:  CF Standard Name Table (v25, 05 July 2013)
    institution:               DOC/NOAA/NESDIS > U.S. Department of Commerce,...
    project:                   GOES
    ...                        ...
    time_coverage_start:       2019-01-02T18:00:36.3Z
    time_coverage_end:         2019-01-02T18:11:14.1Z
    timeline_id:               ABI Mode 3
    production_data_source:    Realtime
    id:                        3ad0c10b-5583-45dd-a158-199addc5b811
    _STRATOPY_:                Metadata(orbit_type='geostationary', platform=...

Download a file from CloudSat.

In [11]:
from unittest import mock

with mock.patch("paramiko.RSAKey.from_private_key_file", return_value="pkey"):
    with mock.patch("paramiko.SSHClient.connect"):
        cs_obj = cloudsat.CloudSat(
        product_type="2B-CLDCLASS.P1_R05",
        username="fakeusr@gmail.edu",
        keyfile="some_file",
        )
        
        csat_data = cs_obj.fetch("2019 dec 12 15:00")

AttributeError: 'NoneType' object has no attribute 'open_sftp_client'

In [12]:
CLOUDSAT_PATH = "../../../tests/data/CloudSat/2019002175851_67551_CS_2B-CLDCLASS_GRANULE_P1_R05_E08_F03.hdf"
csat_data = cloudsat.read_hdf4(CLOUDSAT_PATH)

Merge products

In [14]:
merged_data = mergers.MergePolarGeos("2019 jan 18:30", 
                                     time_zone="UTC", 
                                     trim_size=(3,3)).transform(csat_data, norm_goes)

KeyError: '_STRATOPY_'